<div style="margin-bottom:-35px;">
    <font color=#FFFFFF markdown="1">
        <h1> <center> Gebaseerd op een cursus van:</center> </h1> 
    </font>
    <a href="https://www.aiopschool.be/chatbot/"> 
        <img src="../_afbeeldingen/bannerugentdwengo.png" alt="Dwengo" style ="display: block; margin-left: auto; margin-right: auto; margin-bottom: 30px; width:20%"/>
    </a>
</div>

# Toepassing: ML (lerende) Sentimentanalyse

<div style="background-color:#8B8000">
Voor deze notebook te starten. Zorg zeker dat je de inhoud van 2_AI_sentiment.ipynb begrijpt. In deze notebook zal je dezelfde concepten gebruiken. Nu zal echter een ML-model gebruikt worden om de vervelende stappen uit 2_AI_sentiment.ipynb te automatiseren.
</div>

## 1. Algemene info bij Notebook

### 1.1 Modules installeren

In deze notebook zal je de module <a href="https://spacy.io/">**spacy**</a> gebruiken. Deze module bevat een Machine Learning taalmodel. We zullen deze gebruiken om bepaalde stappen van het voorverwerken te vereenvoudigen (zie ook *deel 1.3 'Doel ML Sentimentanalyse'*).

<div style="background-color:#8B0000"> 
Wees zeker dat je rechtsboven in de juiste kernel zit vooraleer spacy te installeren.
</div>

In [2]:
# installeren van spacy (dit kan even duren)
!pip3 install spacy
# installeren van Nederlandse bibliotheek
!python -m spacy download nl_core_news_sm

  Using cached zipp-3.15.0-py3-none-any.whl (6.8 kB)
  Using cached annotated_types-0.5.0-py3-none-any.whl (11 kB)
  Using cached importlib_metadata-6.7.0-py3-none-any.whl (22 kB)
  Using cached click-8.1.7-py3-none-any.whl (97 kB)


ERROR: pydantic-core 2.14.6 has requirement typing-extensions!=4.7.0,>=4.6.0, but you'll have typing-extensions 4.4.0 which is incompatible.
ERROR: pydantic 2.5.3 has requirement typing-extensions>=4.6.1, but you'll have typing-extensions 4.4.0 which is incompatible.
ERROR: Could not install packages due to an EnvironmentError: [WinError 2] The system cannot find the file specified: 'c:\\venv\\venv_test\\Lib\\site-packages\\srsly\\_json_api.py'

You should consider upgrading via the 'c:\venv\venv_test\scripts\python.exe -m pip install --upgrade pip' command.


  Using cached spacy-3.7.4-cp37-cp37m-win_amd64.whl (12.4 MB)
  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl (29 kB)
  Using cached typer-0.9.4-py3-none-any.whl (45 kB)
  Using cached srsly-2.4.8-cp37-cp37m-win_amd64.whl (482 kB)
  Using cached wasabi-1.1.2-py3-none-any.whl (27 kB)
  Using cached numpy-1.21.6-cp37-cp37m-win_amd64.whl (14.0 MB)
  Using cached thinc-8.2.3-cp37-cp37m-win_amd64.whl (1.5 MB)
  Using cached tqdm-4.66.4-py3-none-any.whl (78 kB)
  Using cached weasel-0.3.4-py3-none-any.whl (50 kB)
  Using cached click-8.1.7-py3-none-any.whl (97 kB)
  Using cached blis-0.7.11-cp37-cp37m-win_amd64.whl (6.6 MB)
  Using cached confection-0.1.4-py3-none-any.whl (35 kB)
  Using cached cloudpathlib-0.16.0-py3-none-any.whl (45 kB)


You should consider upgrading via the 'c:\venv\venv_test\scripts\python.exe -m pip install --upgrade pip' command.


✔ Download and installation successful
You can now load the package via spacy.load('nl_core_news_sm')


You should consider upgrading via the 'c:\venv\venv_test\Scripts\python.exe -m pip install --upgrade pip' command.


### 1.2 Herhaling AI Sentimentanalyse

In de vorige notebook, 'AI sentimentanalyse', maakte je kennis met de principes van een regelgebaseerde (AI) sentimentanalyse: 

 -  Het maakt gebruik van een **lexicon** of woordenboek met daarin woorden gekoppeld aan hun **sentiment** (positief, negatief of neutraal).
 -  Voor een sentimentanalyse uit te voeren op een tekst, moet de tekst eerst **voorverwerkt** worden. Veelvoorkomende preprocessing stappen zijn **lowercasing**, **tokenisering**, **woordsoort tagging** en **lemmatisering**.
 - De sentimentanalyse overloopt de combinatie van woordsoort/lemma voor ieder woord in de tekst. Zit deze in het lexicon? Tel dan de sentimentscore op bij een teller.
 
In de vorige notebook waren nog niet alle stappen geautomatiseerd. **Lemmatisering en woordsoort tagging** moesten manueel gebeuren. Zoals te zien is op onderstaand stappenplan.

<div>
<img src="../_afbeeldingen/AI_sentiment.png" alt="Banner" style ="display: block; margin-left: auto; margin-right: auto; margin-bottom: 10px; width:80%"/>
</div>

### 1.3 Doel ML Sentimentanalyse

In deze notebook zal je de uitvoer van de sentimentanalyse volledig <b>automatiseren</b>. Je zal m.a.w. de computer al het werk laten doen: de computer zal de data preprocessen (voorverwerken) met het <em>machine learning-model (ML-model) Spacy</em>. Vervolgens zal het via een <em>regelgebaseerd AI-systeem</em> de <b>lemmas/woordsoorten</b> matchen in het lexicon. Tenslotte neemt de AI op basis van de score een beslissing over het sentiment van de gegeven tekst. Onderstaand stappenplan toont het nieuwe proces.

<div>
<img src="../_afbeeldingen/ML_sentiment.png" alt="Banner" style ="display: block; margin-left: auto; margin-right: auto; margin-bottom: 10px; width:80%"/>
</div>

## 2. Preprocessing met Spacy

### 2.1 Modules & variabelen klaarzetten


Onderstaande code importeert spacy in deze Notebook. Vervolgens geven we aan welke taalpakket we in spacy willen laden. nl_core_news_sm is het kleinste taalpakket dat voor Nederlands beschikbaar is. Maar ondanks dit heeft het een accuraatheid van 93%!. Anderen zijn <a href="https://spacy.io/usage/models">HIER</a> te vinden.  

In [3]:
# importeren van spacy
import spacy
# inladen van Nederlandse bibliotheek
nlp = spacy.load("nl_core_news_sm")

Naast het model zetten we ook opnieuw de review klaar die we willen voorverwerken.

In [4]:
review = "Nieuw concept in Gent, maar dat kan volgens mij toch beter. De meeste cornflakes waren gewoon de basic soorten. Ook wat duur voor de hoeveelheid die je krijgt, vooral met de toppings zijn ze zuinig. En als je ontbijt aanbiedt, geef de mensen dan toch ook wat meer keuze voor hun koffie."

<div style="background-color:#8B8000">
Het lijkt misschien valspelen dat we andermans ML-model gebruiken. Maar onthoud dat je als programmeur vooral praktisch (lees lui) moet zijn. Het opstellen van een eigen ML-model voor tekstherkenning is mogelijk, maar zou ons veel tijd kosten. Je moet namelijk over heel veel gelabelde data bezitten. Bedenk dat je voor miljoenen woorden een lemma en woordsoort moet opstellen...
</div>

### 2.2 Lowercasing

<div style="background-color:#8B0000"> 
Merk op dat deze stap identiek is aan 'deel 4.2 Lowercasing' van de vorige notebook.
</div>

De aller eerste stap is het omzetten van `review` naar kleine letters. Gelukkig is dit ingebouwd in Python! Zet de tekst in de variabele `review` om naar kleine letters (via welke methode?). Sla het resultaat op in de variabele `review_kleineletters`.

In [6]:
# zet tekst van de review om naar tekst in kleine letters
review_kleineletters = review.lower()

# toon resultaat van lowercasing (er mag geen drukletter meer in de review staan)
print(review_kleineletters)

nieuw concept in gent, maar dat kan volgens mij toch beter. de meeste cornflakes waren gewoon de basic soorten. ook wat duur voor de hoeveelheid die je krijgt, vooral met de toppings zijn ze zuinig. en als je ontbijt aanbiedt, geef de mensen dan toch ook wat meer keuze voor hun koffie.


### 2.3 Tokenisering

Vervolgens moeten we alle woorden in de tekst splitsen en aan een lijst toevoegen.

In voorgaande notebook is de **tokenisering** regelgebaseerd uitgevoerd. het **spacy**-model zal dit echter ook voor zijn rekening nemen (samen met **woordsoort tagging** en **lemmatisering**)! Voer onderstaand code-blok uit.

In [7]:
# review met kleine letters aan model voeren
review_tokens = nlp(review_kleineletters)

Dit was alles. In `review_tokens` kan je de tokens van de review vinden.

<div style="background-color:#8B8000"> 
Merk op dat de leestekens er nu wel bijstaan. Dit is geen probleem. Spacy zal deze ook een correcte woordsoort & lemma geven.
</div>

<div style="background-color:#8B0000"> 
De print maakt deze notebook zeer lang. Duw bovenaan op de knop 'clear all outputs' om deze terug te verwijderen.
</div>

In [8]:
# Print het token zelf
for token in review_tokens:
    print(token.text)

nieuw
concept
in
gent
,
maar
dat
kan
volgens
mij
toch
beter
.
de
meeste
cornflakes
waren
gewoon
de
basic
soorten
.
ook
wat
duur
voor
de
hoeveelheid
die
je
krijgt
,
vooral
met
de
toppings
zijn
ze
zuinig
.
en
als
je
ontbijt
aanbiedt
,
geef
de
mensen
dan
toch
ook
wat
meer
keuze
voor
hun
koffie
.


### 2.4 Woordsoort tagging en Lemmatisering

In `review_tokens` zitten niet alleen de **tokens**, maar ook reeds de **woordsoorten** & **lemma's**!

In [9]:
# review met kleine letters aan model voeren
review_tokens = nlp(review_kleineletters)

In [10]:
# Print de woordsoort tag van elk token
for token in review_tokens:
    print(token.pos_)

ADJ
NOUN
ADP
PROPN
PUNCT
CCONJ
PRON
AUX
ADP
PRON
ADV
ADJ
PUNCT
DET
ADV
NOUN
AUX
ADJ
DET
NOUN
NOUN
PUNCT
ADV
PRON
ADJ
ADP
DET
NOUN
PRON
PRON
VERB
PUNCT
ADV
ADP
DET
NOUN
AUX
PRON
ADJ
PUNCT
CCONJ
SCONJ
PRON
NOUN
VERB
PUNCT
VERB
DET
NOUN
ADV
ADV
ADV
PRON
DET
NOUN
ADP
PRON
NOUN
PUNCT


In [11]:
# Print de lemma van elk token
for token in review_tokens:
    print(token.lemma_)

nieuw
concept
in
Gent
,
maar
dat
kunnen
volgens
mij
toch
goed
.
de
veel
cornflakes
zijn
gewoon
de
basic
soort
.
ook
wat
duur
voor
de
hoeveelheid
die
je
krijgen
,
vooral
met
de
toppings
zijn
ze
zuinig
.
en
als
je
ontbijt
aanbieden
,
geven
de
mens
dan
toch
ook
wat
veel
keuze
voor
hun
koffie
.


<div style="background-color:#8B8000"> 
Merk op dat <b>spacy</b> de woordsoort in het Engels geeft. Dit is ook de reden waarom het **Lexicon** Engelse afkortingen gebruikt.
</div>

#### Oefen mee 2.1:

Doorloop `review_tokens` en stel twee lijsten op `review_lemmas` en `review_soort`.
- De lijst `review_lemmas` bevat alle lemma's (in correcte volgorde).
- De lijst `review_soort` bevat de woordsoorten van alle tokens (in correcte volgorde).

Print deze lijsten na het opstellen.

Als voorbeeld:<br>
- `review_voorverwerkt` = "gent , maar dat kan" <br>
- `review_lemmas` = ["Gent", ",", "maar", "dat", "kunnen"] <br>
- `review_soort` = ["PROPN", "PUNCT", "CCONJ", "PRON", "AUX]

In [12]:
review_lemmas = []
review_soort  = [] 

# VUL AAN

for token in review_tokens:
    review_lemmas.append(token.lemma_)
    review_soort.append(token.pos_)


# Print aangevuld lijsten
print(review_lemmas)
print(review_soort)


['nieuw', 'concept', 'in', 'Gent', ',', 'maar', 'dat', 'kunnen', 'volgens', 'mij', 'toch', 'goed', '.', 'de', 'veel', 'cornflakes', 'zijn', 'gewoon', 'de', 'basic', 'soort', '.', 'ook', 'wat', 'duur', 'voor', 'de', 'hoeveelheid', 'die', 'je', 'krijgen', ',', 'vooral', 'met', 'de', 'toppings', 'zijn', 'ze', 'zuinig', '.', 'en', 'als', 'je', 'ontbijt', 'aanbieden', ',', 'geven', 'de', 'mens', 'dan', 'toch', 'ook', 'wat', 'veel', 'keuze', 'voor', 'hun', 'koffie', '.']
['ADJ', 'NOUN', 'ADP', 'PROPN', 'PUNCT', 'CCONJ', 'PRON', 'AUX', 'ADP', 'PRON', 'ADV', 'ADJ', 'PUNCT', 'DET', 'ADV', 'NOUN', 'AUX', 'ADJ', 'DET', 'NOUN', 'NOUN', 'PUNCT', 'ADV', 'PRON', 'ADJ', 'ADP', 'DET', 'NOUN', 'PRON', 'PRON', 'VERB', 'PUNCT', 'ADV', 'ADP', 'DET', 'NOUN', 'AUX', 'PRON', 'ADJ', 'PUNCT', 'CCONJ', 'SCONJ', 'PRON', 'NOUN', 'VERB', 'PUNCT', 'VERB', 'DET', 'NOUN', 'ADV', 'ADV', 'ADV', 'PRON', 'DET', 'NOUN', 'ADP', 'PRON', 'NOUN', 'PUNCT']


<div style="background-color:#8B0000"> 
Ga pas verder als de printen volgend resultaat geven:
</div>


In [ ]:
review_lemma = ['nieuw', 'concept', 'in', 'Gent', ',', 'maar', 'dat', 'kunnen', 'volgens', 'mij', 'toch', 'goed', '.', 'de', 'veel', 'cornflakes', 'zijn', 'gewoon', 'de', 'basic', 'soort', '.', 'ook', 'wat', 'duur', 'voor', 'de', 'hoeveelheid', 'die', 'je', 'krijgen', ',', 'vooral', 'met', 'de', 'toppings', 'zijn', 'ze', 'zuinig', '.', 'en', 'als', 'je', 'ontbijt', 'aanbieden', ',', 'geven', 'de', 'mens', 'dan', 'toch', 'ook', 'wat', 'veel', 'keuze', 'voor', 'hun', 'koffie', '.']
review_soort = ['ADJ', 'NOUN', 'ADP', 'PROPN', 'PUNCT', 'CCONJ', 'PRON', 'AUX', 'ADP', 'PRON', 'ADV', 'ADJ', 'PUNCT', 'DET', 'ADV', 'NOUN', 'AUX', 'ADJ', 'DET', 'NOUN', 'NOUN', 'PUNCT', 'ADV', 'PRON', 'ADJ', 'ADP', 'DET', 'NOUN', 'PRON', 'PRON', 'VERB', 'PUNCT', 'ADV', 'ADP', 'DET', 'NOUN', 'AUX', 'PRON', 'ADJ', 'PUNCT', 'CCONJ', 'SCONJ', 'PRON', 'NOUN', 'VERB', 'PUNCT', 'VERB', 'DET', 'NOUN', 'ADV', 'ADV', 'ADV', 'PRON', 'DET', 'NOUN', 'ADP', 'PRON', 'NOUN', 'PUNCT']

## 3. Sentimentanalyse

### 3.1 Modules & Variabelen klaarzetten

Gebruik onderstaande code-blok om `lexicon` vanuit lexicondict.json in te laden.

In [13]:
# Lexicon inlezen. Deze bevindt zich in lexicondict.json.
import json
with open("lexicondict.json", "rb") as file: 
    lexicon = json.load(file)

Onderstaande code-blok maakt ook `spacy` klaar voor gebruik.

In [14]:
# importeren van spacy
import spacy
#inladen van Nederlandse bibliotheek
nlp = spacy.load("nl_core_news_sm")

Tenslotte de review die we willen onderzoeken.

In [15]:
review = "Nieuw concept in Gent, maar dat kan volgens mij toch beter. De meeste cornflakes waren gewoon de basic soorten. Ook wat duur voor de hoeveelheid die je krijgt, vooral met de toppings zijn ze zuinig. En als je ontbijt aanbiedt, geef de mensen dan toch ook wat meer keuze voor hun koffie."

### 3.2 Sentimentanalyse uitvoeren

Gebruik alle kennis die je in deze en de vorige notebook hebt opgedaan. Voer een sentimentanalyse op de `review`, gebruik makend van **spacy**.

<div>
<img src="../_afbeeldingen/ML_sentiment.png" alt="Banner" style ="display: block; margin-left: auto; margin-right: auto; margin-bottom: 10px; width:80%"/>
</div>

In [21]:
# Voer hier de sentimentanalyse met spacy uit.
sentiment_score = 0

review_tokens = nlp(review_kleineletters)

review_lemmas = []
review_soort  = [] 
for token in review_tokens:
    review_lemmas.append(token.lemma_)
    review_soort.append(token.pos_)

# VUL VERDER AAN
sentiment_score = 0

aantal = len(review_lemmas)

for i in range(aantal):
    if review_lemmas[i] in lexicon: 
       if review_soort[i] in lexicon[review_lemmas[i]]:
          print(f"{review_lemmas[i]} - {review_soort[i]} : {lexicon[review_lemmas[i]][review_soort[i]]}")
          sentiment_score = sentiment_score + lexicon[review_lemmas[i]][review_soort[i]]
          continue
    
    print(f"{review_lemmas[i]} - {review_soort[i]} : combinatue bestaat niet!")


    
    

# Print de bekomen sentiment score
print(sentiment_score)

nieuw - ADJ : 0.575
concept - NOUN : 0
in - ADP : combinatue bestaat niet!
Gent - PROPN : combinatue bestaat niet!
, - PUNCT : combinatue bestaat niet!
maar - CCONJ : combinatue bestaat niet!
dat - PRON : combinatue bestaat niet!
kunnen - AUX : 1.0
volgens - ADP : combinatue bestaat niet!
mij - PRON : combinatue bestaat niet!
toch - ADV : combinatue bestaat niet!
goed - ADJ : 0.775
. - PUNCT : combinatue bestaat niet!
de - DET : combinatue bestaat niet!
veel - ADV : combinatue bestaat niet!
cornflakes - NOUN : combinatue bestaat niet!
zijn - AUX : 0.0
gewoon - ADJ : 0.5
de - DET : combinatue bestaat niet!
basic - NOUN : combinatue bestaat niet!
soort - NOUN : combinatue bestaat niet!
. - PUNCT : combinatue bestaat niet!
ook - ADV : combinatue bestaat niet!
wat - PRON : combinatue bestaat niet!
duur - ADJ : -1.0666666666666667
voor - ADP : 1.0
de - DET : combinatue bestaat niet!
hoeveelheid - NOUN : combinatue bestaat niet!
die - PRON : combinatue bestaat niet!
je - PRON : combinatue be

Heb je de sentimentanalyse af? Dan kan je onderstaande code-cel gebruiken om een zin te printen bij de bekomen `sentiment_score`.

In [22]:
# eindbeslissing voor deze review
if sentiment_score > 0:
    sentiment = "positief"
elif sentiment_score == 0:
    sentiment = "neutraal"
elif sentiment_score < 0:
    sentiment = "negatief"
print(f"De sentimentscore van de review is: {sentiment_score}")
print("Het sentiment van de review is " + sentiment + ".")    

De sentimentscore van de review is: 5.783333333333333
Het sentiment van de review is positief.


#### Oefen mee 3.2:

Bekijk volgende reviews. 

-  Kies er twee uit en lees deze.

-  Welk sentiment koppel jij zelf aan deze reviews? <div style="background-color:#008000">
    - review 1: Negatief
    - review 2: Neutraal
</div>

-  Voer een sentimentanalyse uit met de ontwikkelde code uit oefen mee 3.1. 

-  Vergelijk de sentimentscore met je eigen verwachtingen. Komen ze overeen?  <div style="background-color:#008000">
    - Antwoord: Half.
</div>

> Sunrisewater is net wat België nodig heeft. Met het obesitasprobleem dat toch wel aan een opmars bezig is, kunnen we alle initiatieven gebruiken om de jeugd weer gewoon water te laten drinken in plaats van die Amerikaanse bucht! Het smaakt geweldig en wat nog beter is, is dat je het gewoon op elke straathoek kan vinden! Echt geweldig! Vooral de pink and yellow is ten zeerste aan te raden.

>  Salé & Sucré staat bekend voor zijn super lekkere en originele cocktails, helaas was er geen alcoholvrije variant te verkrijgen. Onze BOB van dienst moest het dan maar bij frisdrank houden.

>  Het was superleuk om eens te mogen proeven van de Filipijnse keuken. De gerechten zaten goed in elkaar, de porties waren zeker groot genoeg en de smaken zaten helemaal goed. Voor herhaling vatbaar!

>  Gezellige sfeer, lekkere koffie en een mooi interieur. De combinatie van een studiebar en een babbelbar is een geniaal idee! Studeren met een lekker bakkie koffie, een overheerlijk hapje en samen met andere studenten, werkt enorm motiverend. Het interieur is enorm rustgevend met weinig afleiding, waardoor ik nog nooit zoveel heb kunnen doen!

<div style="background-color:#8B8000">
Proficiat, je hebt jouw regelgebaseerd (AI) systeem gecombineerd met een lerend (ML) systeem. Hierdoor is de sentimentanalyse volledig geautomatiseerd!
</div>

#
  <a href="https://www.aiopschool.be/chatbot/"> 
        <img src="../_afbeeldingen/bannerugentdwengo.png" alt="Dwengo" style ="display: block; margin-left: auto; margin-right: auto; margin-bottom: 30px; width:20%"/>
    </a>

Deze Notebook is gebaseerd op: Notebook Chatbot, zie <a href="http://www.aiopschool.be">AI Op School</a>, van S. Pletinck , F. wyffels & N. Gesquière is in licentie gegeven volgens een <a href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Naamsvermelding-NietCommercieel-GelijkDelen 4.0 Internationaal-licentie</a>. 